In [1]:
import pandas as pd
import numpy as np

In [2]:
import requests
import datetime

# Function to get weather data
def get_weather(latitude, longitude, date, time, variables):
	# Format the API URL
	url = f"https://historical-forecast-api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&hourly={','.join(variables)}&start_date={date}&end_date={date}&timezone=UTC"

	# Make the API request
	response = requests.get(url)
	data = response.json()
	# Extract the desired time
	datetime_str = f"{date}T{time}"
	for i, hourly_time in enumerate(data["hourly"]["time"]):
		if hourly_time == datetime_str:
			# Extract requested variables for the specific time
			return {var: data["hourly"][var][i] for var in variables}
	
	return None  

# Example usage
latitude = -34.92
longitude = 138.6
date = "2023-01-01"
time = "06:00"  
variables = ["temperature_2m", "relative_humidity_2m", "windspeed_10m", "apparent_temperature"]

weather_data = get_weather(latitude, longitude, date, time, variables)
print(weather_data)

{'temperature_2m': 32.0, 'relative_humidity_2m': 31, 'windspeed_10m': 14.4, 'apparent_temperature': 32.4}


In [3]:
matches = pd.read_csv('../data/all_years_nc_tc_elo_aggr_stats/matches.csv')

In [4]:
matches.head()[['Date', 'time', 'latitude', 'longitude']]

,Date,time,latitude,longitude
0,2023-01-01,06:25,-34.928181,138.599931
1,2023-01-01,09:40,-34.928181,138.599931
2,2023-01-02,01:40,-34.928181,138.599931
3,2023-01-02,05:15,-34.928181,138.599931
4,2023-01-02,06:00,-34.928181,138.599931


In [5]:
from datetime import datetime, timedelta

def convert_utc_plus1_to_utc(date, time):
    """Convert a UTC+1 time to UTC."""
    # Combine date and time into a single datetime object
    dt = datetime.strptime(f"{date} {time}", "%Y-%m-%d %H:%M")
    # Subtract 1 hour to convert from UTC+1 to UTC
    dt_utc = dt - timedelta(hours=1)
    # Return the date and time in UTC
    return dt_utc.strftime("%Y-%m-%d"), dt_utc.strftime("%H:%M")

def round_time_to_nearest_hour(time_str):
    # Parse the time string into a datetime object
    time_obj = datetime.strptime(time_str, "%H:%M")

    # Round minutes to the nearest hour
    if time_obj.minute >= 30:  # Round up
        time_obj += timedelta(minutes=60 - time_obj.minute)
    else:  # Round down
        time_obj -= timedelta(minutes=time_obj.minute)

    # Return the rounded time in "HH:00" format
    return time_obj.strftime("%H:00")

In [6]:
# Apply get_weather to each row
def fetch_weather_data(row):
	date_utc, time_utc = convert_utc_plus1_to_utc(row["Date"], row["time"])
	rounded_time_utc = round_time_to_nearest_hour(time_utc)
	return get_weather(row["latitude"], row["longitude"], date_utc, rounded_time_utc, variables)

# For now just 100 rows to see if it works
matches = matches.head(100)

weather_data = matches.apply(fetch_weather_data, axis=1)
weather_df = pd.DataFrame(weather_data.tolist())

# Merge the weather data back into the original DataFrame
matches = pd.concat([matches, weather_df], axis=1)
print(matches[['Date', 'time', 'latitude', 'longitude', 'temperature_2m', "relative_humidity_2m", "windspeed_10m", "apparent_temperature"]])

          Date   time   latitude   longitude  temperature_2m  \
0   2023-01-01  06:25 -34.928181  138.599931            32.1   
1   2023-01-01  09:40 -34.928181  138.599931            27.8   
2   2023-01-02  01:40 -34.928181  138.599931            25.7   
3   2023-01-02  05:15 -34.928181  138.599931            26.4   
4   2023-01-02  06:00 -34.928181  138.599931            26.0   
..         ...    ...        ...         ...             ...   
95  2023-01-11  07:40 -36.852095  174.763180            18.2   
96  2023-01-11  09:35 -36.852095  174.763180            18.1   
97  2023-01-11  09:55 -36.852095  174.763180            18.1   
98  2023-01-12  01:40 -34.928181  138.599931            28.1   
99  2023-01-12  04:45 -34.928181  138.599931            29.5   

    relative_humidity_2m  windspeed_10m  apparent_temperature  
0                     33           14.0                  33.5  
1                     47           11.4                  27.9  
2                     53           16.1